# Caracteristica ASTER L1T Rad google Earth Engine

In [1]:
# Importar paquete GEE
import ee
ee.Initialize()

In [2]:
# Agregar un shp GEE
zona_ee = ee.FeatureCollection("users/bravomoralesnino/SHP/ZONA_19S")

In [3]:
# Importar geemap
import geemap
Map = geemap.Map(basemap='ESRI')

In [4]:
geometria = zona_ee.geometry()
Map.centerObject(geometria, 11)

In [5]:
Map.addLayer(zona_ee, {"color" : "00FF11"}, name = "Zona")
Map

Map(center=[-12.908744902436744, -69.9947155021359], controls=(WidgetControl(options=['position', 'transparent…

In [6]:
# Coleccion de ASTER L1T Radiance
ASTER_L1T = ee.ImageCollection("ASTER/AST_L1T_003")\
              .filterDate('2007-01-01','2007-12-31')\
              .filterBounds(zona_ee)\
              .filterMetadata('CLOUDCOVER', 'less_than', 20)

In [7]:
# Conteo de imagenes
count = ASTER_L1T.size().getInfo()
print("Cantidad de imagenes ASTER_L1T:", count)

Cantidad de imagenes ASTER_L1T: 9


## Determinar ID, Fecha, nubosidad, Path y Row

In [8]:
# Imprimir la lista de ID Imagenes
ID_ASTER = ASTER_L1T.reduceColumns(ee.Reducer.toList(),["system:index"]).get("list").getInfo()
print(ID_ASTER)

['20070707145801', '20070714150414', '20070730150418', '20070815150419', '20070831150418', '20070909145805', '20070909145814', '20070925145800', '20071119150343']


In [9]:
# Funcion para determinar fecha imagen
def list_coll(coll):
    def func_ejv(im):
        return ee.String(ee.Image(im).date().format('YYYY/MM/dd')).slice(0)
    return coll.toList(coll.size(), 0).map(func_ejv)

In [10]:
fecha = list_coll(ASTER_L1T).getInfo()
print(fecha)

['2007/07/07', '2007/07/14', '2007/07/30', '2007/08/15', '2007/08/31', '2007/09/09', '2007/09/09', '2007/09/25', '2007/11/19']


In [11]:
# Imprimir la lista Porcentaje Nubosidad
porcentaje_nube = ASTER_L1T.reduceColumns(ee.Reducer.toList(),["CLOUDCOVER"]).get("list").getInfo()
print(porcentaje_nube)

[1, 11, 6, 2, 3, 0, 0, 1, 14]


In [12]:
# Imprimir la lista Path y Row
SCENE_PATH = ASTER_L1T.reduceColumns(ee.Reducer.toList(),["SCENE_PATH"]).get("list").getInfo()
SCENE_ROW = ASTER_L1T.reduceColumns(ee.Reducer.toList(),["SCENE_ROW"]).get("list").getInfo()
print(SCENE_PATH)
print(SCENE_ROW)

[2, 3, 3, 3, 3, 2, 2, 2, 3]
[192, 192, 192, 192, 192, 192, 193, 192, 192]


## Exportar tabla ID, Fecha, Porcentaje Nubosidad

In [13]:
# Crear un diccionario con las listas
dic = {"ID": ID_ASTER, "Fecha": fecha,"Porcentaje Nube": porcentaje_nube, "PATH": SCENE_PATH,"ROW":SCENE_ROW}

In [14]:
import pandas as pd

In [15]:
# Crear un DataFrame mediante el diccionario
df1 = pd.DataFrame(data = dic)
print(df1)
print("Tipo de clase: ", type(dic))

               ID       Fecha  Porcentaje Nube  PATH  ROW
0  20070707145801  2007/07/07                1     2  192
1  20070714150414  2007/07/14               11     3  192
2  20070730150418  2007/07/30                6     3  192
3  20070815150419  2007/08/15                2     3  192
4  20070831150418  2007/08/31                3     3  192
5  20070909145805  2007/09/09                0     2  192
6  20070909145814  2007/09/09                0     2  193
7  20070925145800  2007/09/25                1     2  192
8  20071119150343  2007/11/19               14     3  192
Tipo de clase:  <class 'dict'>


In [17]:
import os

In [18]:
ruta_archivos = r"E:\Teledeteccion_GEE\Python\Datos_Espaciales\Resultado_GEE"
os.chdir(ruta_archivos) # Cambiar ruta trabajo
os.getcwd() # Consultar ruta trabajo

'E:\\Teledeteccion_GEE\\Python\\Datos_Espaciales\\Resultado_GEE'

In [19]:
df1.to_csv("file_name_ASTER_L1T.csv") # relative position

## Seleccionar imagen

In [20]:
# Importar imagen ASTER L1T Rad
ASTER_img = ee.Image("ASTER/AST_L1T_003" + "/" + ID_ASTER[2])

In [21]:
# Nombre de las bandas
print(ASTER_img.bandNames().getInfo())

['B01', 'B02', 'B3N', 'B04', 'B05', 'B06', 'B07', 'B08', 'B09', 'B10', 'B11', 'B12', 'B13', 'B14']


In [22]:
# Bandas RS y TB
bandas_Sup = ["B01","B02","B3N","B04","B05","B06","B07","B08","B09"]
bandas_tem = ["B10","B11","B12","B13","B14"]

In [23]:
# Seleccionar las bandas
ASTER_B123456789 = ASTER_img.select(bandas_Sup)
ASTER_B10_11_12_13_14 = ASTER_img.select(bandas_tem)

## Metadato ASTER L1T

In [24]:
# Propiedades de informacion
ASTER_B123456789.propertyNames().getInfo()

['system:version',
 'ORIGINAL_BANDS_PRESENT',
 'system:id',
 'PRODUCTION_TIME',
 'GAIN_COEFFICIENT_B10',
 'AVERAGE_CORRELATION_COEFFICIENT_B05',
 'BAD_PIXELS_B09',
 'GAIN_COEFFICIENT_B11',
 'AVERAGE_CORRELATION_COEFFICIENT_B06',
 'BAD_PIXELS_B08',
 'GAIN_COEFFICIENT_B12',
 'BAD_PIXELS_B07',
 'GAIN_COEFFICIENT_B13',
 'AVERAGE_CORRELATION_COEFFICIENT_B04',
 'BAD_PIXELS_B06',
 'GAIN_COEFFICIENT_B14',
 'BAD_PIXELS_B01',
 'GAIN_COEFFICIENT_B08',
 'GAIN_COEFFICIENT_B09',
 'BAD_PIXELS_B05',
 'BAD_PIXELS_B04',
 'system:footprint',
 'BAD_PIXELS_B02',
 'SWIR_POINTING_ANGLE',
 'SATELLITE_REVOLUTION_NUMBER',
 'RESAMPLING_METHOD_B11',
 'RESAMPLING_METHOD_B10',
 'RESAMPLING_METHOD_B13',
 'GAIN_SETTING_B09',
 'RESAMPLING_METHOD_B12',
 'GAIN_SETTING_B08',
 'GAIN_SETTING_B07',
 'RESAMPLING_METHOD_B14',
 'GAIN_SETTING_B06',
 'GAIN_SETTING_B05',
 'GAIN_SETTING_B04',
 'GAIN_COEFFICIENT_B01',
 'SCENE_VIEW',
 'GAIN_COEFFICIENT_B02',
 'GAIN_SETTING_B02',
 'GAIN_SETTING_B01',
 'SOLAR_AZIMUTH',
 'GAIN_COEFFICI

In [25]:
# Determinar coeficiente ganancia B1
ASTER_img.get('GAIN_COEFFICIENT_B01').getInfo()

0.676

In [26]:
# Determinar angulo elevacion solar
SOLAR_ELEVATION = ASTER_img.get('SOLAR_ELEVATION').getInfo()
SOLAR_ELEVATION

49.541722

In [27]:
import math
SOLAR_ELEVATION_R = SOLAR_ELEVATION*math.pi/180
SOLAR_ELEVATION_R

0.8646661660077102

In [28]:
# Simbologia ND
viz = {
    'bands': ['B07', 'B3N', 'B02'],
    'min': 30,
    'max': 150,
    'gamma': 1.8
}

In [29]:
# Visualizar 
Map.addLayer(ASTER_B123456789, viz,name = "ASTER L1T Rad")
Map

Map(bottom=281405.0, center=[-12.908744902436744, -69.9947155021359], controls=(WidgetControl(options=['positi…

## Recortar imagen ASTER L1T

In [30]:
# Recortar
ASTER_B123456789_clip = ASTER_B123456789.clip(zona_ee)

In [31]:
# Visualizar 
Map.addLayer(ASTER_B123456789_clip, viz,name = "ASTER L1T Rad")
Map

Map(bottom=140889.0, center=[-12.957720912158363, -69.96162414550783], controls=(WidgetControl(options=['posit…